In [9]:
import tensorflow as tf
from tensorflow import keras

In [11]:
model = keras.models.load_model("/data/study/git/pytorch_cook/04_trt/data/b1.h5")

2022-06-08 09:19:47.967026: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-08 09:19:51.964955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6
2022-06-08 09:19:52.032516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6


In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 62, 1)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 96, 62, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
1 (Normalization)               (None, 96, 62, 1)    3           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 97, 63, 1)    0           1[0][0]                          
______________________________________________________________________________________________

In [13]:
model.save('/data/study/git/pytorch_cook/04_trt/data/model_pb')

2022-06-08 09:21:13.394029: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /data/study/git/pytorch_cook/04_trt/data/model_pb/assets


/root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [14]:
import numpy as np

BATCH_SIZE = 1024

dummy_input_batch = np.zeros((BATCH_SIZE, 96, 62, 1))

In [15]:
import tensorrt as trt
from tensorflow.python.compiler.tensorrt import trt_convert as tf_trt
from tensorflow.python.saved_model import tag_constants

In [16]:
PRECISION = "FP32" # Options are "FP32", "FP16", or "INT8"

In [19]:
from trt_help import ModelOptimizer # using the helper from <URL>

model_dir = '/data/study/git/pytorch_cook/04_trt/data/model_pb'

opt_model = ModelOptimizer(model_dir)

In [20]:
model_fp32 = opt_model.convert(model_dir+'_FP32', precision=PRECISION)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


2022-06-08 09:25:30.277225: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2022-06-08 09:25:30.277369: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-06-08 09:25:30.284500: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6
2022-06-08 09:25:30.285624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6
2022-06-08 09:25:30.513875: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 2259 nodes (1814), 3290 edges (2845)

INFO:tensorflow:Assets written to: /data/study/git/pytorch_cook/04_trt/data/model_pb_FP32/assets


INFO:tensorflow:Assets written to: /data/study/git/pytorch_cook/04_trt/data/model_pb_FP32/assets


In [21]:
model_fp32.predict(dummy_input_batch)

2022-06-08 09:26:19.886256: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-08 09:26:24.050602: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-06-08 09:26:27.991185: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-08 09:26:32.345166: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(1024, 28), dtype=float32, numpy=
array([[2.7626586e-05, 9.5862607e-03, 2.6109561e-04, ..., 1.7746914e-05,
        7.8309407e-05, 3.3253562e-04],
       [2.7626586e-05, 9.5862607e-03, 2.6109561e-04, ..., 1.7746914e-05,
        7.8309407e-05, 3.3253562e-04],
       [2.7626586e-05, 9.5862607e-03, 2.6109561e-04, ..., 1.7746914e-05,
        7.8309407e-05, 3.3253562e-04],
       ...,
       [2.7626586e-05, 9.5862607e-03, 2.6109561e-04, ..., 1.7746914e-05,
        7.8309407e-05, 3.3253562e-04],
       [2.7626586e-05, 9.5862607e-03, 2.6109561e-04, ..., 1.7746914e-05,
        7.8309407e-05, 3.3253562e-04],
       [2.7626586e-05, 9.5862607e-03, 2.6109561e-04, ..., 1.7746914e-05,
        7.8309407e-05, 3.3253562e-04]], dtype=float32)>

In [22]:
# Warm up - the first batch through a model generally takes longer
model.predict(dummy_input_batch)
model_fp32.predict(dummy_input_batch)

ValueError: in user code:

    /root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1584 predict_function  *
        return step_function(self, iterator)
    /root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1574 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /root/miniconda3/envs/tf26_trt/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /root/miniconda3/env